In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
from modelfcts.backgrounds import generate_odorant
from modelfcts.ideal import find_projector, find_parallel_component
from utils.metrics import jaccard
from modelfcts.tagging import create_sparse_proj_mat, project_neural_tag

In [ ]:
def ideal_inhib_samples(dimensions, nsamples, projmat, rates, conc, rng, proj_kwargs):
    """ Dimensions: n_r, n_b, n_k
    nsamples: n_backs, n_new"""
    # Generate new odors
    n_r, n_b, n_k = dimensions
    n_backs, n_new = nsamples
    lambda_in = 1.0 / rates
    backgrounds = generate_odorant(n_rec=[n_backs, n_b, n_r], rgen=rng, lambda_in=lambda_in)
    new_odors = generate_odorant(n_rec=[n_new, n_r], rgen=rng, lambda_in=lambda_in)
    jaccard_scores = np.zeros(nsamples)
    perp_elements = np.zeros(list(nsamples) + [n_r])
    new_odor_tags = []
    flag = True
    # Find new odor tags
    for j in range(n_new):
        new_odor_tags.append(project_neural_tag(new_odors[j], new_odors[j], projmat, **proj_kwargs))
    # Loop over backgrounds
    for i in range(n_backs):
        projector = find_projector(backgrounds[i].T)
        for j in range(n_new):
            x_par = find_parallel_component(new_odors[j], basis=None, projector=projector)
            # Project new odor, project its perpendicular component
            perp_tag = project_neural_tag(conc*(new_odors[j] - x_par), new_odors[j], projmat, **proj_kwargs) 
            jaccard_scores[i, j] = jaccard(new_odor_tags[j], perp_tag)
            # Distribution of elements after subtracting parallel component.
            perp_elements[i, j] = x_par
            if np.any(x_par < 0.0) and flag:
                flag = False
                fig = plt.figure()
                ax = fig.add_subplot(111, projection='3d')
                ax.quiver(*([np.zeros(2)]*3), *(backgrounds[i, :2, :3].T), color="k")
                ax.plot(*(backgrounds[i].T), color="k", marker="o", ls="none", ms=1)
                ax.quiver(*([np.zeros(1)]*3), *new_odors[j, :3], color="r")
                ax.plot(*new_odors[j], color="r", marker="o", ms=1, ls="none")
                ax.quiver(*([np.zeros(1)]*3), *x_par[:3], color="xkcd:burgundy")
                ax.plot(*x_par, color="xkcd:burgundy", marker="o", ms=1, ls="none")
                plt.show()
                plt.close()
    return jaccard_scores, perp_elements

In [ ]:
# Initialize parameters
main_rgen = np.random.default_rng(seed=0x45e3746c789a8fe24df70a2980a83a33)
layer_dimensions = [25, 6, 1000]
number_samples = [10, 60, 60]
projection_kwargs = {}  # using defaults
average_element = 10.0
new_conc = 0.5

In [ ]:
# Try 100 different projection matrices
all_jaccards = np.zeros(number_samples)
all_proj_mats = []
all_perp_vecs = np.zeros(list(number_samples) + [layer_dimensions[0]])
for i in range(number_samples[0]):
    proj_mat = create_sparse_proj_mat(layer_dimensions[2], layer_dimensions[0], main_rgen, fraction_filled=6/50)
    all_proj_mats.append(proj_mat)
    all_jaccards[i], all_perp_vecs[i] = ideal_inhib_samples(
            layer_dimensions, number_samples[1:], proj_mat, 
            average_element, new_conc, main_rgen, projection_kwargs
            )

In [ ]:
plt.close()

## Jaccard distribution

In [ ]:
sp.stats.skew(all_jaccards.flatten())

In [ ]:
# Try to fit a gamma distribution
# dist, data, bounds=None, *, guess=None, method='mle', optimizer=<function differential_evolution>
# No, gamma is no good
#fitres = sp.stats.fit(sp.stats.gamma, all_jaccards.flatten(), 
#                      bounds={"a":[0, 100], "scale":[0.0001, 100.0], "loc":[0, 0]})
fitres = sp.stats.fit(sp.stats.norm, all_jaccards.flat, bounds={"loc":[0.0, 1.0], "scale":[0.05, 1.0]})
print(fitres)

In [ ]:
fig, ax = plt.subplots()
ax.hist(all_jaccards.flatten(), bins=30, density=True)
xrange = np.arange(0.0, 1.0, 0.01)
ax.plot(xrange, sp.stats.norm.pdf(xrange, loc=fitres.params.loc, scale=fitres.params.scale))
#ax.plot(xrange, sp.stats.beta.pdf(xrange, a=fitres.params.a, b=fitres.params.b))
ax.set(xlabel=r"Jaccard($z_n, z_{n\perp}$)", 
      ylabel="Prob. density")
plt.show()
plt.close()

## Perpendicular component vectors element distribution

In [ ]:
fig, ax = plt.subplots()
ax.hist(all_perp_vecs.flatten(), bins=200, density=False)
#xrange = np.arange(0.0, 1.0, 0.01)
#ax.plot(xrange, sp.stats.norm.pdf(xrange, loc=fitres.params.loc, scale=fitres.params.scale))
#ax.plot(xrange, sp.stats.beta.pdf(xrange, a=fitres.params.a, b=fitres.params.b))
ax.set(xlabel=r"$\vec{x}_{n\perp}$ elements magnitude", 
      ylabel="Prob. density")
plt.show()
plt.close()

## Intersection cardinality distribution

In [ ]:
cardinalities_int = np.round(0.1*layer_dimensions[2] * all_jaccards / (1.0 + all_jaccards)).astype(int)

In [ ]:
#fitres2 = sp.stats.fit(sp.stats.binom, cardinalities_int.flat, 
#                       bounds={"n":[30, 51], "p":[0.05, 1.0]})
# Negative hypergeometric would make sense. Sampling without replacement from a population
# in which K are considered successes. Sampling until until r failures are encountered
# The successes are the KCs belonging to the new odor tag. 
# Parameters: M, n, r
#fitres2 = sp.stats.fit(sp.stats.nhypergeom, cardinalities_int.flat[::100], 
#                       bounds={"M":[50, 100], "n":[45, 55], "r":[1, 60]})

# Try beta-binomial instead. Binomial where the prob of success follows beta distribution
# Makes more sense even. 
fitres2 = sp.stats.fit(sp.stats.betabinom, cardinalities_int.flat[::100], 
                       bounds={"n":[40, 100], "a":[0, 10], "b":[0, 10]})
#fitres2 = sp.stats.fit(sp.stats.nchypergeom_fisher, cardinalities_int.flat[::100], 
#                      bounds={"M":[45, 300], "n":[30, 80], "N":[45, 55], "odds":[0, 1]})

In [ ]:
print(fitres2)

In [ ]:
# Distribution of card(zn, znperp)
#cardinalities = 0.1*layer_dimensions[2] * all_jaccards / (1.0 + all_jaccards)
fig, ax = plt.subplots()
ax.hist(cardinalities_int.flat, bins=51, density=True)
xrange = np.arange(fitres2.params.n+1)
ax.plot(xrange, sp.stats.betabinom.pmf(xrange, *fitres2.params))
#ax.plot(xrange, sp.stats.nchypergeom_fisher.pmf(xrange, *fitres2.params))
ax.set(xlabel=r"card($z_n \cap z_{n\perp}$)", ylabel="Prob. density")
plt.show()
plt.close()